In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

## Load data

In [2]:
data = pd.read_csv('data/cards.csv')    
prices = pd.read_csv('data/prices.csv')

pd.options.display.max_columns = data.shape[1] #display option
# data.describe(include='all')

C:\Users\Przemyslaw\.conda\envs\mtg\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6,9,12,16,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
prices_no_duplicates = prices.drop_duplicates(subset=['uuid'])
# prices_no_duplicates.describe(include='all')

In [4]:
# prices.head()
# data.head()
data.columns

Index(['index', 'id', 'artist', 'asciiName', 'borderColor', 'colorIdentity',
       'colorIndicator', 'colors', 'convertedManaCost', 'duelDeck',
       'edhrecRank', 'faceConvertedManaCost', 'flavorName', 'flavorText',
       'frameEffect', 'frameEffects', 'frameVersion', 'hand', 'hasFoil',
       'hasNoDeckLimit', 'hasNonFoil', 'isAlternative', 'isArena', 'isBuyABox',
       'isDateStamped', 'isFullArt', 'isMtgo', 'isOnlineOnly', 'isOversized',
       'isPaper', 'isPromo', 'isReprint', 'isReserved', 'isStarter',
       'isStorySpotlight', 'isTextless', 'isTimeshifted', 'layout',
       'leadershipSkills', 'life', 'loyalty', 'manaCost', 'mcmId', 'mcmMetaId',
       'mtgArenaId', 'mtgoFoilId', 'mtgoId', 'multiverseId', 'name', 'names',
       'number', 'originalText', 'originalType', 'otherFaceIds', 'power',
       'printings', 'purchaseUrls', 'rarity', 'scryfallId',
       'scryfallIllustrationId', 'scryfallOracleId', 'setCode', 'side',
       'subtypes', 'supertypes', 'tcgplayerProduc

In [5]:
data_merged = pd.merge(data, prices_no_duplicates, on='uuid', how='inner')

In [6]:
data_merged.drop_duplicates(subset='name', keep="last", inplace=True)

### Include selected features

In [7]:
# len(data_merged)
data_merged = data_merged[['name', 'text', 'price', 'convertedManaCost']]
# print(data_merged.dtypes)

In [8]:
data_merged[data_merged.price > 100][['name', 'text', 'price', 'convertedManaCost']]

,name,text,price,convertedManaCost
5003,Ali from Cairo,Damage that would reduce your life total to le...,304.95,4.0
5006,Bazaar of Baghdad,"{T}: Draw two cards, then discard three cards.",1499.99,0.0
5012,City in a Bottle,Whenever one or more other nontoken permanents...,399.95,2.0
5021,Diamond Valley,"{T}, Sacrifice a creature: You gain life equal...",250.99,0.0
5022,Drop of Honey,"At the beginning of your upkeep, destroy the c...",419.96,1.0
...,...,...,...,...
34102,Ravages of War,Destroy all lands.,179.92,4.0
34182,Zodiac Dragon,When Zodiac Dragon is put into your graveyard ...,243.20,9.0
35944,Grim Tutor,Search your library for a card and put that ca...,231.87,3.0
39121,"Ugin, the Spirit Dragon","[+2]: Ugin, the Spirit Dragon deals 3 damage t...",138.21,8.0


### Zamiania typu na string dla tokenizera

In [9]:
data_merged['text'] = data_merged['text'].astype(str)

### Tokenizacja

In [105]:
vocab_size = 1000 #cannot be too long for lstm layer
max_length = data_merged.text.map(lambda x: len(x)).max()
# max_length = 100
trunc_type='post'
padding_type='post'

In [106]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(data_merged['text'])
sequences = tokenizer.texts_to_sequences(data_merged['text'])

In [107]:
# tokenizer.word_index

### Padding

In [108]:
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
# padded

### Attach converted mana cost

In [109]:
data_merged['convertedManaCost'].to_numpy().reshape(-1, 1)

array([[4.],
       [4.],
       [5.],
       ...,
       [5.],
       [2.],
       [5.]])

In [110]:
padded = np.append(padded, data_merged['convertedManaCost'].to_numpy().reshape(-1, 1), axis=1)
max_length = max_length + 1
# padded = data_merged['convertedManaCost'].to_numpy().reshape(-1, 1)
# max_length = 1

### Preprocess prices

In [111]:
# prices.describe()
# prices.head()

In [112]:
prices_filtered = data_merged['price']

In [113]:
prices_np = prices_filtered.to_numpy()

### train, test, validation split (80, 10, 10)

In [114]:
X_train, X_test, y_train, y_test = train_test_split(padded, prices_np, test_size=0.2)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

### Standarize

In [115]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

y_scaler = StandardScaler()
y_train = y_scaler.fit_transform(y_train.reshape(-1, 1))
y_test = y_scaler.transform(y_test.reshape(-1, 1))
y_val = y_scaler.transform(y_val.reshape(-1, 1))

## Model

In [122]:
model = tf.keras.models.Sequential([
#     tf.keras.layers.Embedding(vocab_size, 256, input_length=max_length),
#     tf.keras.layers.GlobalAveragePooling1D(),
#     tf.keras.layers.LSTM(512, batch_input_shape=(max_length, 1, 1), return_sequences=False),
    tf.keras.layers.Embedding(vocab_size, 256, input_length=max_length),
    tf.keras.layers.LSTM(256, return_sequences=True),
#     tf.keras.layers.SimpleRNN(units=256),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.Dense(512, activation='relu'),
#     tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1)
])

model.summary()

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_25 (Embedding)     (None, 770, 256)          256000    
_________________________________________________________________
lstm_34 (LSTM)               (None, 770, 256)          525312    
_________________________________________________________________
flatten_6 (Flatten)          (None, 197120)            0         
_________________________________________________________________
dense_87 (Dense)             (None, 1)                 197121    
Total params: 978,433
Trainable params: 978,433
Non-trainable params: 0
_________________________________________________________________


### Tensorboard

In [123]:
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)

In [124]:
optimizer = tf.keras.optimizers.RMSprop(0.001)
model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])

In [125]:
X_train[0].shape,max_length, vocab_size

((770,), 770, 1000)

### Train

In [126]:
epochs = 500
batch_size = 128
model.fit(X_train, y_train, epochs=epochs, callbacks=[tensorboard_callback], validation_data=(X_val, y_val), batch_size=batch_size)

Train on 16521 samples, validate on 2066 samples
Epoch 1/500
16521/16521 [==============================] - 47s 3ms/sample - loss: 5.0914 - mae: 0.6559 - mse: 5.0914 - val_loss: 1.0089 - val_mae: 0.8355 - val_mse: 1.0089
Epoch 2/500
16521/16521 [==============================] - 43s 3ms/sample - loss: 1.7181 - mae: 0.3213 - mse: 1.7181 - val_loss: 0.4418 - val_mae: 0.1347 - val_mse: 0.4418
Epoch 3/500
16521/16521 [==============================] - 43s 3ms/sample - loss: 1.4046 - mae: 0.2921 - mse: 1.4046 - val_loss: 1.0343 - val_mae: 0.7700 - val_mse: 1.0343
Epoch 4/500
16521/16521 [==============================] - 43s 3ms/sample - loss: 1.2730 - mae: 0.2390 - mse: 1.2730 - val_loss: 0.4546 - val_mae: 0.1156 - val_mse: 0.4546
Epoch 5/500
16521/16521 [==============================] - 44s 3ms/sample - loss: 1.1389 - mae: 0.2372 - mse: 1.1389 - val_loss: 0.4484 - val_mae: 0.2089 - val_mse: 0.4484
Epoch 6/500
16521/16521 [==============================] - 44s 3ms/sample - loss: 1.0546 - 

KeyboardInterrupt: 

### read some value

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in tokenizer.word_index.items()])
def decode_sentence(text):
    text = scaler.inverse_transform(text)
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

def decode_price(val):
    return y_scaler.inverse_transform(val)

In [ ]:
for i in range(100):
    price = decode_price(y_train[i])
    if price > 1:
        print(i)
        print(decode_price(y_train[i]))

In [ ]:
print(decode_sentence(X_train[70]))

In [ ]:
data_merged[data_merged['name'].str.contains("Zur the Enchanter")]